In [45]:
!pip install joblib
!pip install pyshp
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
import shapefile
import numpy as np
import gdal

In [2]:

# View the working path
import os
 
print(os.getcwd())
 
 
 # Change the working path
import os
from google.colab import drive
drive.mount('/content/gdrive')
 
path = "/content/gdrive/My Drive/RandomForest"

os.chdir("/content/gdrive/MyDrive/RandomForest")
!pwd
!ls

/content
Mounted at /content/gdrive
/content/gdrive/MyDrive/RandomForest
L8_PALSAR_SUBSET_WGS84.tfw	    TrainingMultipart.dbf
L8_PALSAR_SUBSET_WGS84.tif	    TrainingMultipart.prj
L8_PALSAR_SUBSET_WGS84.tif.aux.xml  TrainingMultipart.sbn
mymodel.gz			    TrainingMultipart.sbx
outMap1.tif			    TrainingMultipart.shp
outMap.tif			    TrainingMultipart.shp.xml
TrainingMultipart.cpg		    TrainingMultipart.shx


In [47]:
shp = shapefile.Reader("TrainingMultipart.shp")

In [48]:
records = np.asarray(shp.records())

Next line will list the first record of the shape file.  The first attribute is the class, the second attribute is a value coming from the first band of the RS stack, and so on.

In [49]:
records[0]

array([ 3,  7, 61, 27, 71, 96, 86])

In [50]:
RF_clf = RandomForestClassifier()

Note below, python, to read the bands requires from 1 to 7 eventhough there are only 6 bands.

In [51]:
X = records[:,1:7]
y = records[:,0]

In [52]:
X.shape

(1853, 6)

In [53]:
RF_clf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [54]:
param_grid ={"n_estimators": [500],
                             "max_features": ['sqrt', 'log2'],                                                
                             "max_depth": [10, None],
                             "min_samples_split": [2,3,5],
                             "min_samples_leaf": [5,10,20,50,100,200,500],
                             "bootstrap": [True, False]}

In [55]:
grid = GridSearchCV(RF_clf, param_grid=param_grid, 
                                    cv=StratifiedKFold(5), n_jobs=-1,
                                    scoring='accuracy')

In [56]:
grid.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,...
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_s

In [57]:
grid.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [58]:
grid.best_score_

0.6556887885189772

In [59]:
model_gz = grid.best_estimator_

In [60]:
joblib.dump(model_gz, 'mymodel.gz')

['mymodel.gz']

In [61]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  6 19:19:14 2017

@author: Ciaran 

function classify pixel block reads in a raster, classifies it with a 
pre-existing model (eg C:/pathto/model.gz) then writes out to a new image

The default output format is geotiff so you don't need to add the '.tif' part



"""
import numpy as np
import gdal
from tqdm import tqdm #this gives a progress bar install with pip


def classify_pixel_bloc(model, inputImage, bands, outMap, blocksize=256, 
                        FMT=None, dtype = gdal.GDT_Int32):
    """
    A block processing classifier for large rasters, supports KEA, HFA, & Gtiff
    formats. KEA is recommended, Gtiff is the default
    
    Where:
        model = a path to a scikit learn model that has been saved as a '.gz'
            file
        inputImage = path to image including the file fmt 'Myimage.tif'
        bands = the no of image bands eg 8
        outMap = path to output image excluding the file format 'pathto/mymap'
        FMT = optional parameter - gdal readable fmt
        blocksize = optional - size of raster chunck 256 tends to be quickest
                    if you put None it will read size from gdal (this doesn't
                    always pay off!)
        dtype = optional gdal dataype - default is int32
    Usage (typical - leaving defaults)
    
    classify_pixel_block(model, inputImage, 8, outMap)
        
    """
    if FMT == None:
        FMT = 'Gtiff'
        fmt = '.tif'
    if FMT == 'HFA':
        fmt = '.img'
    if FMT == 'KEA':
        fmt = '.kea'
    if FMT == 'Gtiff':
        fmt = '.tif'
    
    inDataset = gdal.Open(inputImage)
    x_pixels = inDataset.RasterXSize  # number of pixels in x
    y_pixels = inDataset.RasterYSize  # number of pixels in y
    geotransform = inDataset.GetGeoTransform()
    PIXEL_SIZE = geotransform[1]  # size of the pixel...they are square so thats ok.
    #if not would need w x h
    x_min = geotransform[0]
    y_max = geotransform[3]
    # x_min & y_max are like the "top left" corner.
    projection = inDataset.GetProjection()
    geotransform = inDataset.GetGeoTransform()   
    #dtype=gdal.GDT_Int32
    driver = gdal.GetDriverByName(FMT)
    
    # Set params for output raster
    outDataset = driver.Create(
        outMap+fmt, 
        x_pixels,
        y_pixels,
        1,
        dtype)

    outDataset.SetGeoTransform((
        x_min,    # 0
        PIXEL_SIZE,  # 1
        0,                      # 2
        y_max,    # 3
        0,                      # 4
        -PIXEL_SIZE))
        
    outDataset.SetProjection(projection)    
    band = inDataset.GetRasterBand(1)
    cols = inDataset.RasterXSize
    rows = inDataset.RasterYSize
    outBand = outDataset.GetRasterBand(1)
    PIXEL_SIZE = geotransform[1] 
    # So with most datasets blocksize is a row scanline
    if blocksize == None:
        blocksize = band.GetBlockSize()
        blocksizeX = blocksize[0]
        blocksizeY = blocksize[1]
    else:
        blocksizeX = blocksize
        blocksizeY = blocksize

    # For either option below making a block index is FAR slower than the 
    # code used below - don't be tempted - likely cython is the solution to 
    # performance gain here or para processing (but the model is already multi-core)
    
    
    model1 = joblib.load(model)
    if blocksizeY==1:
        rows = np.arange(y_pixels, dtype=np.int)                
        for row in tqdm(rows):
            i = int(row)
            j = 0
            #X = np.zeros(shape = (bands, blocksizeX))
            #for band in range(1,bands+1):
            X = inDataset.ReadAsArray(j, i, xsize=blocksizeX, ysize=blocksizeY)
            X.shape = ((bands,blocksizeX))
            
            if X.max() == 0:
                predictClass = np.zeros_like(rows, dtype = np.int32)
            else: 
                X = np.where(np.isfinite(X),X,0) # this is a slower line                
                X = X.transpose() 
                #Xs = csr_matrix(X)
                predictClass = model1.predict(X)
                predictClass[X[:,0]==0]=0 
            
            outBand.WriteArray(predictClass.reshape(1, blocksizeX),j,i) 
                    #print(i,j)

    # else it is a block            
    else:
        for i in tqdm(range(0, rows, blocksizeY)):
            if i + blocksizeY < rows:
                numRows = blocksizeY
            else:
                numRows = rows -i
        
            for j in range(0, cols, blocksizeX):
                if j + blocksizeX < cols:
                    numCols = blocksizeX
                else:
                    numCols = cols - j
                X = inDataset.ReadAsArray(j,i, xsize=numCols, ysize=numRows)
#                X = np.zeros(shape = (bands, numCols*numRows))
#                for band in range(1,bands+1):
#                    band1 = inDataset.GetRasterBand(band)
#                    data = band1.ReadAsArray(j, i, numCols, numRows)
                if X.max() == 0:
                    continue              
                else:
                    X.shape = ((bands,numRows*numCols))
                    X = X.transpose() 
                    X = np.where(np.isfinite(X),X,0) # this is a slower line   
                    #Xs= csr_matrix(X)
                    predictClass = model1.predict(X)
                    predictClass[X[:,0]==0]=0                    
                    predictClass = np.reshape(predictClass, (numRows, numCols))
                    outBand.WriteArray(predictClass,j,i)
                #print(i,j)
    outDataset.FlushCache()
    outDataset = None

def prob_pixel_bloc(model, inputImage, bands, outMap, classes, blocksize=256,
                    FMT=None):
    """
    The same inputs as above but outputs a probability image where each band is a
    class probability
    
    This is slower than thematic version!
    
    classes = no of classes eg 10
    """
    if FMT == None:
        FMT = 'Gtiff'
        fmt = 'tif'
    if FMT == 'HFA':
        fmt = '.img'
    if FMT == 'KEA':
        fmt = '.kea'
    if FMT == 'Gtiff':
        fmt = '.tif'
    
    inDataset = gdal.Open(inputImage)
    x_pixels = inDataset.RasterXSize  # number of pixels in x
    y_pixels = inDataset.RasterYSize  # number of pixels in y
    geotransform = inDataset.GetGeoTransform()
    PIXEL_SIZE = geotransform[1]  # size of the pixel...they are square so thats ok.
    #if not would need w x h
    x_min = geotransform[0]
    y_max = geotransform[3]
    # x_min & y_max are like the "top left" corner.
    projection = inDataset.GetProjection()
    geotransform = inDataset.GetGeoTransform()   
    dtype=gdal.GDT_Float64
    driver = gdal.GetDriverByName(FMT)
    
    # Set params for output raster
    outDataset = driver.Create(outMap+fmt, x_pixels, y_pixels, classes, dtype)

    outDataset.SetGeoTransform((
        x_min,    # 0
        PIXEL_SIZE,  # 1
        0,                      # 2
        y_max,    # 3
        0,                      # 4
        -PIXEL_SIZE))
        
    outDataset.SetProjection(projection)    
    band = inDataset.GetRasterBand(1)
    cols = inDataset.RasterXSize
    rows = inDataset.RasterYSize
    #outBand = outDataset.GetRasterBand(1)
    PIXEL_SIZE = geotransform[1] 
    # So with most datasets blocksize is a row scanline
    if blocksize == None:
        blocksize = band.GetBlockSize()
        blocksizeX = blocksize[0]
        blocksizeY = blocksize[1]
    else:
        blocksizeX = blocksize
        blocksizeY = blocksize
     # size of the pixel...they are square so thats ok.
    #if not would need w x h
    #If the block is a row, this simplifies things a bit
    # Key issue now is to speed this part up 
    # For either option below making a block index is FAR slower than the 
    # code used below - don't be tempted - likely cython is the solution to 
    # performance gain here
    model1 = joblib.load(model)
    if blocksizeY==1:
        rows = np.arange(y_pixels, dtype=np.int)        
        for row in tqdm(rows):

            i = int(row)
            j = 0
            X = np.zeros(shape = (bands, blocksizeX))
            for band in range(1,bands+1):
                band1 = inDataset.GetRasterBand(band)
                data = band1.ReadAsArray(j, i, blocksizeX, 1)
                X[band-1] = data.flatten()
            X = X.transpose() 
            X = np.where(np.isfinite(X),X,0) # this is a slower line   
            Probs=model1.predict_proba(X)

            ProbsFinal = np.empty((data.shape[0], data.shape[1], classes))
            for band in range(1,classes+1):
                ProbsFinal[:,:,band-1] = np.reshape(Probs[:,band-1], (1, blocksizeX))
                outBand = outDataset.GetRasterBand(band)
                outBand.WriteArray(ProbsFinal[:,:,band-1],j,i)
                #print(i,j)

    # else it is a block            
    else:
        model1 = joblib.load(model)
        for i in tqdm(range(0, rows, blocksizeY)):
            if i + blocksizeY < rows:
                numRows = blocksizeY
            else:
                numRows = rows -i
        
            for j in range(0, cols, blocksizeX):
                if j + blocksizeX < cols:
                    numCols = blocksizeX
                else:
                    numCols = cols - j
                X = np.zeros(shape = (bands, numCols*numRows))
                for band in range(1,bands+1):
                    band1 = inDataset.GetRasterBand(band)
                    data = band1.ReadAsArray(j, i, numCols, numRows)
                    X[band-1] = data.flatten()
                X = X.transpose() 
                X = np.where(np.isfinite(X),X,0)  # this is a slower line  
                Probs=model1.predict_proba(X)
                #Class 1 = Deforestattion
                #if inputImage.RasterXSize < inputImage.RasterYSize:
                #classArr = np.arange(classes)
                ProbsFinal = np.empty((data.shape[0], data.shape[1], classes))
                for band in range(1,classes+1):
                    ProbsFinal[:,:,band-1] = np.reshape(Probs[:,band-1], (data.shape[0], data.shape[1]))
                    outBand = outDataset.GetRasterBand(band)
                    outBand.WriteArray(ProbsFinal[:,:,band-1],j,i)
                    #print(i,j)
        

    #outBand.FlushCache()
    outDataset.FlushCache()
    outDataset = None

In [62]:
classify_pixel_bloc('mymodel.gz', 'L8_PALSAR_SUBSET_WGS84.tif', 6, 'outMap1', blocksize=256, 
                        FMT=None, dtype = gdal.GDT_Int32)

100%|██████████| 12/12 [10:47<00:00, 53.95s/it]
